In [20]:
import sys
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Cleaning the data

In [21]:
df = pd.read_csv('../data/listings.csv');
df = df[['id', 'description', 'host_is_superhost', 'zipcode', 'property_type','room_type', 'bathrooms', 'bedrooms',
    'beds','price', 'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people', 'review_scores_rating']]

### Drop unknown zipcodes and unwanted property types

In [22]:
df = df.dropna(subset=['zipcode'])
df['zipcode'] = df['zipcode'].str.extract('(07\d{3})')
value_counts = df['zipcode'].value_counts() 
to_remove = value_counts[value_counts <= 1].index
df['zipcode'].replace(to_remove, np.nan, inplace=True)

df = df[df.room_type == 'Entire home/apt']
acceptedPropTypes = ['Apartment', 'House', 'Villa', 'Chalet', 'Condominium', 'Townhouse', 'Other', 'Loft']
# , 'Bungalow', 'Guesthouse', 'Cabin', 'Serviced apartment', 'Earth House', 'Nature lodge']
df = df[df.property_type.isin(acceptedPropTypes)]

### Cast the money-values into floats and booleans to proper booleans

In [23]:
def moneyLaundering(df, c):
    newCol = df[c].astype(str).apply(lambda x: (x.replace('$', '')))
    newCol = newCol.astype(str).apply(lambda x: (x.replace(',', '')))
    newCol = newCol.astype(str).apply(lambda x: (x.replace('nan', '0')))
    df = df.drop([c], axis=1)
    df[c] = pd.to_numeric(newCol.values, errors='raise')
    return df

df = moneyLaundering(df, 'cleaning_fee')
df = moneyLaundering(df, 'security_deposit')
df = moneyLaundering(df, 'price')
df = moneyLaundering(df, 'extra_people')

df['host_is_superhost'] = df['host_is_superhost'].str.contains('t', regex=False)
df.host_is_superhost = df.host_is_superhost.apply(lambda x: x*1)
# Use "has secutiry_deposit" instead of security_deposit
# df['security_deposit'] = np.where(df['security_deposit']>0, True, False)

###  Adjust price, normalize it and bucket it into lowCost, Medium and Premium

In [24]:
df['price'] = df['price'] + df['cleaning_fee']
df['guests_not_included'] = df['beds'] - df['guests_included']
mask = df['guests_not_included'] < 0 
df.loc[mask, 'guests_not_included'] = 0
price = df['price'] + df['guests_not_included']*df['extra_people']
df['price'] =  pd.to_numeric(price.values, errors='raise')

# remove top 5% most expensive and most cheap
l, h = np.nanpercentile(df['price'], [5, 95])
df = df[df['price'] < h]
df = df[df['price'] > l]

# norm price
df['normPrice'] = np.log(df['price'])

### Get features from description (seaViews, hasBalcony, hasSwimmingpool)

In [25]:
seaViews = df['description'].str.contains('vistas al mar|sea view|seaview|mit Meeresblick|Blick aufs Meer', regex=True)
df['seaViews'] = seaViews.apply(lambda x: x*1)
balcony = df['description'].str.contains('terraza|balcón|terrace|balcony', regex=True)
df['balcony'] = balcony.apply(lambda x: x*1)
swimmingPool = df['description'].str.contains('piscina|swimmingPool', regex=True)
df['swimmingPool'] = swimmingPool.apply(lambda x: x*1)

descLength = df['description'].str.len()
df['descLength'] = descLength

### Extract the availability feature

In [26]:
dfcal = pd.read_csv('../data/15_03_2017_calendar.csv');
dfcal = dfcal[dfcal.listing_id.isin(df.id)]

In [27]:
temp = dfcal[dfcal.available == 't']
temp = temp.groupby(['listing_id'], as_index=False).agg({'available': 'count'})
temp['availability'] = temp.available.apply(lambda x: x*100/365)
temp.describe()
df = df.set_index('id').join(temp.set_index('listing_id'))

### Create a dataframe dropping all the values missing reviews

In [28]:
reviewed = df.dropna(subset=['review_scores_rating'])

# Select the wanted properties and save it into different datasets

In [29]:
# Some usefull prints before saving the data
# df.describe(include='all')
# df['price'].max()
# df.count()
# df.sample(10)

##### General data (features + groupedPrice label)

In [30]:
general_data = df[['zipcode', 'property_type', 'bathrooms', 'bedrooms', 'beds', 'host_is_superhost', 'swimmingPool',
                'balcony', 'security_deposit', 'seaViews', 'availability', 'review_scores_rating', 'descLength']]

a, b, c, d = np.nanpercentile(df['price'], [0, 33, 67, 100])
general_data['groupedPrice'] = pd.cut(df['price'], [a, b, c, d], labels=['lowCost', 'medium', 'premium'])

general_data.to_csv(path_or_buf= '../data/general_data.csv', index=False)
general_data.describe()

,bathrooms,bedrooms,beds,host_is_superhost,swimmingPool,balcony,security_deposit,seaViews,availability,review_scores_rating,descLength
count,11013.000000,11012.000000,11020.000000,11019.000000,11017.000000,11017.000000,11020.000000,11017.000000,10801.000000,5979.000000,11017.000000
mean,2.014801,2.770251,4.296189,0.071241,0.093038,0.484615,154.767151,0.067532,64.533099,91.940626,799.215576
std,1.034704,1.268751,2.145054,0.257238,0.290499,0.499786,185.691279,0.250952,24.555976,9.095364,307.048008
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,1.000000
25%,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,47.000000,89.000000,563.000000
50%,2.000000,3.000000,4.000000,0.000000,0.000000,0.000000,100.000000,0.000000,68.000000,94.000000,1000.000000
75%,2.500000,4.000000,6.000000,0.000000,0.000000,1.000000,300.000000,0.000000,84.000000,100.000000,1002.000000
max,8.000000,10.000000,16.000000,1.000000,1.000000,1.000000,4000.000000,1.000000,100.000000,100.000000,1819.000000


### Model with all the appartments reviewed

In [31]:
general_reviewed_data = reviewed[['zipcode', 'property_type', 'bathrooms', 'bedrooms', 'beds', 'host_is_superhost',
                                  'swimmingPool', 'balcony', 'security_deposit', 'seaViews', 'availability',
                                  'review_scores_rating', 'descLength']]

a, b, c, d = np.nanpercentile(df['normPrice'], [0, 33, 67, 100])
general_reviewed_data['groupedPrice'] = pd.cut(df['normPrice'], [a, b, c, d], labels=['lowCost', 'medium', 'premium'])

general_reviewed_data.to_csv(path_or_buf= '../data/general_reviewed_data.csv', index=False)
general_reviewed_data.describe()

l = general_reviewed_data.zipcode.count()
s = int((0.8*general_reviewed_data.zipcode.count()))
reviewed_train = general_reviewed_data.iloc[0:s]
reviewed_test = general_reviewed_data.iloc[s:l-1]

reviewed_train.to_csv(path_or_buf= '../data/reviewed_train.csv', index=False)
reviewed_test.to_csv(path_or_buf= '../data/reviewed_test.csv', index=False)

##### Normalized general data (features + groupedPrice label)

In [32]:
norm_general_data = df[['zipcode', 'property_type', 'bathrooms', 'bedrooms', 'beds', 'host_is_superhost', 'swimmingPool',
                'balcony', 'security_deposit', 'seaViews', 'availability', 'review_scores_rating', 'descLength']]

a, b, c, d = np.nanpercentile(df['normPrice'], [0, 33, 67, 100])
norm_general_data['groupedPrice'] = pd.cut(df['normPrice'], [a, b, c, d], labels=['lowCost', 'medium', 'premium'])

norm_general_data.to_csv(path_or_buf= '../data/norm_general_data.csv', index=False)

##### Split into train and test datasets

In [33]:
l = norm_general_data.zipcode.count()
s = int((0.8*norm_general_data.zipcode.count()))
train = norm_general_data.iloc[0:s]
test = norm_general_data.iloc[s:l-1]

train.to_csv(path_or_buf= '../data/norm_train.csv', index=False)
test.to_csv(path_or_buf= '../data/norm_test.csv', index=False)

##### All the data for visualization

In [34]:
norm_all_data = df.drop('description', axis=1)
norm_all_data['groupedPrice'] = norm_general_data['groupedPrice']
norm_all_data.to_csv(path_or_buf= '../data/norm_all_data.csv', index=False)

#### Label groupedPrice depends on totalPrice/nGuests

In [35]:
label_price_per_guess = general_data

total_guests = df['guests_not_included']+df['guests_included']
price_per_guest = df['price']/total_guests

l, h = np.nanpercentile(price_per_guest, [5, 95])
price_per_guest = price_per_guest[price_per_guest < h]
price_per_guest = price_per_guest[price_per_guest > l]
a, b, c, d = np.nanpercentile(price_per_guest, [0, 33, 67, 100])

label_price_per_guess['groupedPrice'] = pd.cut(price_per_guest, [a, b, c, d], labels=['lowCost', 'medium', 'premium'])

label_price_per_guess.to_csv(path_or_buf= '../data/label_price_per_guess.csv', index=False)

##### Split into train and test datasets

In [36]:
l = label_price_per_guess.zipcode.count()
s = int((0.8*label_price_per_guess.zipcode.count()))
train = label_price_per_guess.iloc[0:s]
test = label_price_per_guess.iloc[s:l-1]

train.to_csv(path_or_buf= '../data/label_price_per_guess_train.csv', index=False)
test.to_csv(path_or_buf= '../data/label_price_per_guess_test.csv', index=False)

In [37]:
# send roman
norm_general_data = df[['zipcode', 'property_type', 'bathrooms', 'bedrooms', 'beds', 'host_is_superhost', 'swimmingPool',
                'balcony', 'security_deposit', 'seaViews', 'availability', 'review_scores_rating', 'descLength']]

a, b, c, d = np.nanpercentile(df['normPrice'], [0, 33, 67, 100])
norm_general_data['groupedPrice'] = pd.cut(df['normPrice'], [a, b, c, d], labels=['lowCost', 'medium', 'premium'])

norm_general_data.to_csv(path_or_buf= '../data/norm_general_data.csv', index=False)